# Spark 


* how to run using docker image for spark

    * options is the course docker image: ucsddse230/cse255-dse230
 
```sh
docker run --name edx_big_data -it -p 8889:8888 -v /media/leandroohf/sdb1/leandro/edx_big_data_analytics_using_spark:/home/ucsddse230/ ucsddse230/cse255-dse230 /bin/bash

# If you need to ssh to the container
docker exec -it edx_big_data /bin/bash

```
    * serach a simple docker image in docker hub


* Basic operations



In [4]:
import numpy as np
import pandas as pd

from scipy import stats

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

import IPython
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sqlalchemy import create_engine
import datetime as dt


## Creation

## Transfortmations



* You cannot use any operation on the map functio. The operation should NOT depend of the other like subtraction or division. Will get different results while runnning multiple times

In [ ]:
# groupByKey return (key, <iterator>)

A = sc.parallelize([(1,3), (3,100),(1,-5),(3,2)])
A.groupByKey().mapValues(lambda x: [elem for elem in x ])

# output
#[ (1, [3,-5]), (3, [100, 2]) ]

#  countByKey: returns dictionary
A = sc.parallelize([(1,3), (3,100),(1,-5),(3,2)])

A.countByKey()

# output (dictionnary
# {1:2, 3:2}

# lookup (key): returns the list of all of the values associated with key
A = sc.parallelize([(1,3), (3,100),(1,-5),(3,2)])

A.lookup(3)

# output (list)
# [100,2]

#  collectAsMap(): like collect() - collect returns list of tuples -  but returns a map = Dictionary
A = sc.parallelize([(1,3), (3,100),(1,-5),(3,2)])
A.collectAsMap()

# output Dictionary
# {1:[3,-5], 3: [100,2]}

In [ ]:
## Famous word count example (hello word)

words = ['this', 'is', 'the', 'best', 'mac', 'ever']

wordRDD = sc.parallelize(words)

wordRDD.reduce(lambda w,v: w if len(w) < len(v) else v)

# another example is sum (the prder does not matter)
B=sc.parallelize([1,3,5,2])

B.reduce(lambda x,y: x+y)


## Actions